In [ ]:
# Imports
import pandas as pd
import numpy as np
import sys
sys.path.append('..')

from load_data import load_dataset
import google.generativeai as genai
from insights.prompt_templates import *
import json
from datetime import datetime
import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
# Configure Gemini
genai.configure(api_key=os.environ.get("GEMINI_API_KEY"))
model = genai.GenerativeModel('gemini-pro')

print("Gemini model loaded successfully")

In [ ]:
# Load Reddit Data
reddit_df = load_dataset('reddit')
print(f"Loaded {len(reddit_df)} Reddit posts")
print(f"Columns: {list(reddit_df.columns)}")

# Sample the data to see text content
print("\nSample post:")
print(reddit_df[['title', 'text', 'subreddit', 'score']].iloc[0])